In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

ICM_all_dataframe = pd.read_csv("Dataset/data_ICM_metadata.csv")
URM_all_dataframe = pd.read_csv("Dataset/data_train.csv")

In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

ICM_all = sps.coo_matrix((ICM_all_dataframe['data'], (ICM_all_dataframe['item_id'], ICM_all_dataframe['feature_id'])))
ICM_all = ICM_all.tocsr()

URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

#ICM_train_validation, ICM_test = split_train_in_two_percentage_global_sample(ICM_all, train_percentage=0.8)
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [7]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

stacked_URM = sps.vstack([URM_all, ICM_all.T])
# RP3
rp3_recommender = RP3betaRecommender(stacked_URM)
rp3_recommender.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325)

RP3betaRecommender: Similarity column 38121 (100.0%), 914.18 column/sec. Elapsed time 41.70 sec


In [8]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(rp3_recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/rp3_stacked_samebestparam.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


# Okay, maybe hyperparameter tuning?
Let's avoid k fold for the moment

In [4]:
import os
os.chdir('../')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
ICM_all_dataframe = pd.read_csv("Dataset/data_ICM_metadata.csv")


print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [6]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

ICM_all = sps.coo_matrix((ICM_all_dataframe['data'], (ICM_all_dataframe['item_id'], ICM_all_dataframe['feature_id'])))

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [7]:
stacked_URM_train = sps.vstack([URM_train_validation, ICM_all.T])

In [9]:
import optuna
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

def objective_function(trial):
    params = {
        'topK': trial.suggest_int('topK', 5, 300),
        'alpha': trial.suggest_float('alpha', 0, 2),
        'beta': trial.suggest_float('beta', 0, 2),
    }
    
    MAP = 0
    recommender = RP3betaRecommender(stacked_URM_train)
    recommender.fit(**params)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP += result_dict.loc[10]["MAP"]
    
    
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/RP3/rp3_stacked.csv", index = False)

optuna_study = optuna.create_study(study_name="rp3_stacked", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 13:02:43,033] A new study created in memory with name: rp3_stacked


RP3betaRecommender: Similarity column 38121 (100.0%), 882.95 column/sec. Elapsed time 43.17 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 17.73 sec. Users per second: 2008


[I 2024-12-30 13:03:48,516] Trial 0 finished with value: 0.018065415153278634 and parameters: {'topK': 252, 'alpha': 0.5090155916828913, 'beta': 1.3211353738564091}. Best is trial 0 with value: 0.018065415153278634.


RP3betaRecommender: Similarity column 38121 (100.0%), 1035.91 column/sec. Elapsed time 36.80 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 14.63 sec. Users per second: 2432


[I 2024-12-30 13:04:41,398] Trial 1 finished with value: 0.03160567294764429 and parameters: {'topK': 43, 'alpha': 0.20252393930250912, 'beta': 0.9203773949673912}. Best is trial 1 with value: 0.03160567294764429.


...

RP3betaRecommender: Similarity column 38121 (100.0%), 1051.51 column/sec. Elapsed time 36.25 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 15.36 sec. Users per second: 2317


[I 2024-12-30 13:53:38,974] Trial 48 finished with value: 0.03931559408424027 and parameters: {'topK': 28, 'alpha': 0.8987595727247959, 'beta': 0.7536232061275119}. Best is trial 42 with value: 0.050856120553759106.


RP3betaRecommender: Similarity column 38121 (100.0%), 743.49 column/sec. Elapsed time 51.27 sec


[W 2024-12-30 13:54:51,379] Trial 49 failed with parameters: {'topK': 275, 'alpha': 0.47122679548928015, 'beta': 0.2803874761956744} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_77445/1784160990.py", line 15, in objective_function
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = recommender_object.recommend(t

KeyboardInterrupt: 

## Reducing hyperparameter

In [10]:
def objective_function_smaller(trial):
    params = {
        'topK': trial.suggest_int('topK', 5, 100),
        'alpha': trial.suggest_float('alpha', 0, 1),
        'beta': trial.suggest_float('beta', 0, 1),
    }
    
    MAP = 0
    recommender = RP3betaRecommender(stacked_URM_train)
    recommender.fit(**params)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP += result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/RP3/rp3_stacked_smallerhyper.csv", index = False)

optuna_study = optuna.create_study(study_name="rp3_stacked", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_smaller,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 13:55:30,245] A new study created in memory with name: rp3_stacked


RP3betaRecommender: Similarity column 38121 (100.0%), 949.79 column/sec. Elapsed time 40.13 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 15.82 sec. Users per second: 2249


[I 2024-12-30 13:56:28,752] Trial 0 finished with value: 0.032643711106205016 and parameters: {'topK': 97, 'alpha': 0.4351994863080879, 'beta': 0.916718686530137}. Best is trial 0 with value: 0.032643711106205016.


RP3betaRecommender: Similarity column 38121 (100.0%), 1008.48 column/sec. Elapsed time 37.80 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 14.70 sec. Users per second: 2421


[I 2024-12-30 13:57:22,563] Trial 1 finished with value: 0.03356469719975766 and parameters: {'topK': 39, 'alpha': 0.8162417566749208, 'beta': 0.9312235631330348}. Best is trial 1 with value: 0.03356469719975766.


...

[I 2024-12-30 15:23:31,807] Trial 97 finished with value: 0.0495410169967675 and parameters: {'topK': 7, 'alpha': 0.3304529580883502, 'beta': 0.3638646304923815}. Best is trial 92 with value: 0.05135146723389284.


RP3betaRecommender: Similarity column 38121 (100.0%), 1037.45 column/sec. Elapsed time 36.74 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 13.44 sec. Users per second: 2649


[I 2024-12-30 15:24:22,768] Trial 98 finished with value: 0.051015389177629517 and parameters: {'topK': 16, 'alpha': 0.38463379276369936, 'beta': 0.3084803046201766}. Best is trial 92 with value: 0.05135146723389284.


RP3betaRecommender: Similarity column 38121 (100.0%), 933.82 column/sec. Elapsed time 40.82 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 14.95 sec. Users per second: 2381


[I 2024-12-30 15:25:19,457] Trial 99 finished with value: 0.0504057596817799 and parameters: {'topK': 19, 'alpha': 0.4370630696485556, 'beta': 0.4579907084172361}. Best is trial 92 with value: 0.05135146723389284.


In [11]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.05135146723389284, 'topK': 15.0, 'alpha': 0.4295346473303171, 'beta': 0.29566454493527333}


So, even after tuning, still worse than normal RP3